In [21]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer 
import sys
!{sys.executable} -m pip install xgboost
from xgboost import XGBClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier



In [22]:
X_train = pd.read_csv("train.csv")
X_test = pd.read_csv("test.csv")

In [23]:
X_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [24]:
y_train = X_train["Survived"]
X_train.drop(["PassengerId","Name","Ticket","Survived","Cabin","Parch"],axis = 1,inplace = True)
test_id = X_test["PassengerId"]
X_test.drop(["PassengerId","Name","Ticket","Cabin","Parch"],axis = 1,inplace = True)



In [25]:
# Get list of categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

print("Categorical variables:")
print(object_cols)



Categorical variables:
['Sex', 'Embarked']


In [26]:
OH_encoder = OneHotEncoder(handle_unknown = "ignore",sparse = False)
OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(X_train[object_cols]))
OH_cols_test = pd.DataFrame(OH_encoder.transform(X_test[object_cols]))
OH_cols_train.index = X_train.index
OH_cols_test.index = X_test.index
num_X_train = X_train.drop(object_cols, axis=1)
num_X_test = X_test.drop(object_cols,axis = 1)


In [27]:
my_imputer = SimpleImputer()
imputed_X_train = pd.DataFrame(my_imputer.fit_transform(num_X_train))
imputed_X_test = pd.DataFrame(my_imputer.transform(num_X_test))
imputed_X_train.columns = num_X_train.columns
imputed_X_test.columns = num_X_test.columns

In [28]:
train = pd.concat([imputed_X_train,OH_cols_train],axis = 1)
test = pd.concat([imputed_X_test,OH_cols_test],axis = 1)

In [29]:
clf = XGBClassifier(max_depth = 30,learning_rate = 0.001,n_estimators = 200)
clf.fit(train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.001, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=30, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=200,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [30]:
predictions = pd.DataFrame(clf.predict(test))
test_id = pd.DataFrame(test_id)

In [31]:
scores = cross_val_score(clf, train, y_train, cv=5)
print("Mean cross-validation score: %.2f" % scores.mean())


Mean cross-validation score: 0.83


In [32]:
output = pd.concat([test_id,predictions],axis = 1)

In [33]:
output.rename(columns= {0: "Survived"},inplace = True)

In [34]:
output.to_csv("predictions.csv",index= False)